In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Indices for key landmarks on the iris and eye corners
left_iris_index = 468
left_eye_inner = 133
left_eye_outer = 33
left_eye_top = 159
left_eye_bottom = 145

right_iris_index = 473
right_eye_inner = 362
right_eye_outer = 263
right_eye_top = 386
right_eye_bottom = 374

    
# Function to calculate gaze direction
def is_concentrated(face_landmarks, frame_width, frame_height):
    
    # Get positions of left and right iris centers
   
    left_rye_z = face_landmarks.landmark[left_iris_index].z
    right_iris_z = face_landmarks.landmark[right_iris_index].z
    
    left_eye_y = face_landmarks.landmark[left_eye_outer].y * frame_height
    right_eye_y = face_landmarks.landmark[right_eye_outer].y * frame_height
    eye_y_difference = abs(left_eye_y - right_eye_y)
    
    # Tilted face
    if eye_y_difference > 15: # higher = more angle
        return False
    
    left_iris_x = face_landmarks.landmark[left_iris_index].x * frame_width
    right_iris_x = face_landmarks.landmark[right_iris_index].x * frame_width

    # Get eye corner positions
    left_eye_inner_x = face_landmarks.landmark[left_eye_inner].x * frame_width
    left_eye_outer_x = face_landmarks.landmark[left_eye_outer].x * frame_width
    right_eye_inner_x = face_landmarks.landmark[right_eye_inner].x * frame_width
    right_eye_outer_x = face_landmarks.landmark[right_eye_outer].x * frame_width

    # Calculate relative positions for gaze detection
    left_gaze_ratio = (left_iris_x - left_eye_inner_x) / (left_eye_outer_x - left_eye_inner_x)
    right_gaze_ratio = (right_iris_x - right_eye_inner_x) / (right_eye_outer_x - right_eye_inner_x)
    
    z_diff = abs(left_rye_z - right_iris_z)
    
    # Check the head pose
    if z_diff > 0.028: # head pose (higher = more angle)
        
        if (left_rye_z > right_iris_z): # looking left

            if  left_gaze_ratio > 0.55: # higher = more angle
                 return False
            else:
              return True
            
        else: # looking right
            if  right_gaze_ratio > 0.55: # higher = more angle
                return False
            else:
              return True
            
       
    # Check if both irises are centered within each eye
    if abs(left_gaze_ratio - right_gaze_ratio) < 0.14: # higher = more angle
        return True

    return False


# Start video capture
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Mirror effect for better visualization
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    # If landmarks are detected, calculate gaze and head position
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Calculate gaze direction and head tilt
            concentrated = is_concentrated(face_landmarks, frame_width, frame_height)
            
            if concentrated:
                text = "Concentrated"
                color = (0, 255, 0)
            else:
                text = "Not Concentrated"
                color = (0, 0, 255)

            # Display the gaze direction and head tilt on the frame
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)      

    # Show the frame
    cv2.imshow('Eyeball & Head Pose Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
face_mesh.close()
cv2.destroyAllWindows()